In [7]:
import pandas as pd
import numpy as np
import random
import json
from dataclasses import dataclass
import pygad 
random.seed(42)

# First Step : convert the data into the right datatype

In [8]:
def import_data(file:str): # json file 
    with open(file) as f:
        data = json.load(f)

        # convert to numpy array
        data["preferences"] = np.array(data["preferences"])
        data["authorship"] = np.array(data["authorship"])
        data["friendships"] = np.array(data["friendships"])

    return data


In [9]:
data = import_data("datasets/easy_dataset_1.json")
data

{'dataset_id': 'Easy Dataset 1',
 'num_papers': 5,
 'num_reviewers': 5,
 'reviewer_capacity': 3,
 'min_reviews_per_paper': 3,
 'max_reviews_per_paper': 5,
 'preferences': array([[3, 2, 1, 5, 3],
        [5, 2, 4, 2, 2],
        [4, 3, 1, 2, 4],
        [4, 2, 2, 1, 1],
        [4, 3, 4, 1, 5]]),
 'friendships': array([[0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]),
 'authorship': array([[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 0, 0, 1],
        [0, 0, 1, 1, 0]])}

## Fitness Function: Evaluating Reviewer Assignments

The fitness function is a key component of our genetic algorithm. Its purpose is to quantify the satisfaction of reviewers while also accounting for constraints by penalizing violations. Then maximizing it optimize reviewer preferences but also incorporates important constraints.

---
The fitness function relies on several key matrices and elements (given by the problem): 

In the following, 
   - $ n $ is the number of reviewers.
   - $ m $ is the number of papers.

 **Preference Matrix $ P $**
   - **Purpose**: This matrix captures the satisfaction each reviewer has for each paper.
   - **Description**: Each element $ P[i, j] $ represents how much reviewer $ i $ prefers reviewing paper $ j $. 

$$
P \in \mathbb{N}^{n \times m}, \quad P[i, j] \text{ is the preference of reviewer } i \text{ for paper } j
$$


 **Authorship Matrix $ A_{\text{auth}} $**
   - **Purpose**: This matrix indicates which reviewer is the author of which paper.
   - **Description**: Each element $ A_{\text{auth}}[i, j] $ is $ 1 $ if reviewer $ i $ is the author of paper $ j $, and $ 0 $ if they are not. This is important because we want to avoid authors reviewing their own papers.
$$
A_{\text{auth}} \in \{0, 1\}^{n \times m}, \quad A_{\text{auth}}[i, j] = 1 \text{ if reviewer } i \text{ authored paper } j
$$


 **Friendship Matrix $ F $**
   - **Purpose**: This matrix represents the friendship relationships between reviewers. Friendships are important because two friends should not review the same paper to avoid conflicts of interest.
   - **Description**: Each element $ F[i, j] = 1 $ if reviewers $ i $ and $ j $ are friends, and $ 0 $ if they are not.

$$
F \in \{0, 1\}^{n \times n}, \quad F[i, j] = 1 \text{ if reviewer } i \text{ and reviewer } j \text{ are friends}
$$

**Additional Constraints**
#TODO explain here the others stuff like the max number of paper etc.. 

---
### Structure (datatype) of one solution

To model a solution, we represent each solution as a matrix $A$ ($n \times p$) where each element indicates the assignment of a reviewer to a paper.

$$
A[i, j] = 1 \quad \text{if reviewer } i \text{ is assigned to paper } j, \quad A[i, j] = 0 \quad \text{otherwise.}
$$
___

### The fitness function
**1. Satisfaction Score**

We construct our fitness function using a satisfaction matrix, defined as follows. 
The satisfaction score is computed by:

$$
S = \sum_{i=1}^n \sum_{j=1}^m A[i, j] \cdot P[i, j]
$$

This allows us to capture the satisfaction of reviewers without considering any constraints. Then, step by step, we refine this matrix to incorporate the various constraints. 


**2. Authorship Constraint**

Reviewers cannot review papers they authored. This is penalized using the authorship matrix $ A_{auth} $. The modified satisfaction matrix incorporates this constraint:

$$
S' = S \cdot (1 - \text{author\_weight} \cdot A_{auth})
$$

Here, `author_weight` determines the penalty applied to assignments violating this constraint.



**3. Friendship Constraint**

If two reviewers who are friends review the same paper, their satisfaction scores are reduced. 
For every pair of friends $ (i, k) $:

$$
S''[i, j] = S'[i, j] \cdot \text{friendship\_weight} \quad \text{if} \quad S'[i, j] > 0 \text{ and } S'[k, j] > 0
$$

Here, `friendship_weight` reduces the satisfaction score for shared papers between friends.



 **4. Final Fitness Score**

Finally, to obtain a scalar output from our fitness function, we compute the sum of the adjusted satisfaction matrix S′′
$$
\text{Fitness} = \sum_{i=1}^n \sum_{j=1}^m S''[i, j]
$$


In [13]:
class GeneticAlgorithm:
    def __init__(self, file_path:str):
        self.context = self.import_data(file_path)
        self.ga_instance = None
    

    def import_data(self, file:str): # json file 
        with open(file) as f:
            data = json.load(f)

            # convert to numpy array
            data["preferences"] = np.array(data["preferences"])
            data["authorship"] = np.array(data["authorship"])
            data["friendships"] = np.array(data["friendships"])

        return data

    def fitness(self, agent: np.array, author_weight=1, friendship_weight=0.5):
        """ 
        Returns a score corresponding to the satisfaction the agent obtains with respect to the constraints
        we want him to follow.
        """
        agent = agent.reshape(self.context["preferences"].shape)
        n = self.context["num_reviewers"]
        m = self.context["num_papers"] 

        # Satisfaction matrix: the pleasure each reviewer takes by reviewing it's attributed papers
        satisfaction = agent * self.context["preferences"]

        # Add author constraint
        author_cons = np.ones((n,m)) - author_weight * self.context["authorship"]
        satisfaction = np.multiply(satisfaction, author_cons)

        # Add friendship constraint
        friendship = np.triu(self.context["friendships"])    # only need upper triangular part because matrix is symmetric
        agents1, agents2 = np.where(friendship == 1)

        # Iterate over every pair of friends
        for i1, i2 in zip(agents1, agents2):
            # And then over all the papers they can review
            for j in range(m):
                if satisfaction[i1, j] and satisfaction[i2, j]: # Case where 2 friends review the same paper
                    satisfaction[i1, j] = satisfaction[i1, j]*friendship_weight
                    satisfaction[i2, j] = satisfaction[i2, j]*friendship_weight
        return np.sum(satisfaction)


    def fitness_wrapper(self, ga_instance, solution, solution_idx):
        return self.fitness(solution) # make sur to pass the context and to have define fintness before
    
    def run(self): # here we'll add parameters 
        self.ga_instance =  pygad.GA(num_generations=100,
                                        num_parents_mating=5,
                                        fitness_func=self.fitness_wrapper,
                                        sol_per_pop=20,
                                        num_genes=self.context["num_papers"]*self.context["num_reviewers"],
                                        gene_space=[0,1],
                                        )
        self.ga_instance.run()
        solution, solution_fitness, _ = self.ga_instance.best_solution()

        return solution.reshape(self.context["preferences"].shape), solution_fitness
                                     



In [14]:
#test class 
ga = GeneticAlgorithm("datasets/easy_dataset_1.json")
ga.run()


/Users/luc/miniconda3/envs/intSyst/lib/python3.12/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


(array([[1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 0.],
        [0., 1., 0., 1., 1.],
        [0., 0., 1., 1., 1.],
        [1., 1., 0., 1., 1.]]),
 47.0)

# former code

In [ ]:
def genetic_algo(dataset, nb_agents, mutation_freq):
    """
    Genetic algorithm concerning the distribution of papers reviews.

    Parameters:

    nb_agents (int) : The number of agents who populate the algorithm for each generation
    mutation_freq (float in [0;1]) : Mutation frequence
    """

    assert(0 <= mutation_freq <= 1)

    # Get context via dataset reading
    context = import_data(dataset)
    n = context["num_reviewers"]
    m = context["num_papers"]

    # Randomly generates some agents (= n*m matrices)
    agents = [np.random.rand(n,m).round() for _ in range(nb_agents)]

    scores = [0]
    it = 0
    while(max(scores) < 50 and it < 10):

        # FITNESS COMPUTATION
        scores = [fitness(ag, context, n, m) for ag in agents]
        
        
        # AGENT SELECTION
        new_agents = []
        new_len = int(0.8 * nb_agents)
        
        # Roulette metaphora
        while len(new_agents) < new_len:
            selection_arrow = random.random()   # random value between 0 and 1
            sum_scores = sum(scores)
            roulette_score = 0

            # Fill the roulette little by little
            # If the arrow is in the area we just added, the agent is selected for the next generation
            for i in range(len(agents)):

                pi = scores[i]/sum_scores
                
                # Area is proportional to agent fitness score
                if roulette_score < selection_arrow < roulette_score + pi:
                    new_agents.append(agents.pop(i))
                    scores.pop(i)
                    break

        
        # CROSSOVERS
        while(len(new_agents) < nb_agents):
            k = len(new_agents)

            # Choose two different random agents
            (i, j) = (random.randint(0, k-1), random.randint(0,k-1))
            if i == j:
                continue
            
            # Take half of the two agents, splitted vertically
            split_index = m // 2
            left = new_agents[i][:, :split_index]
            right = new_agents[j][:, split_index:]

            # Glue them to build a new agent
            new_agents.append(np.hstack((left, right)))


        # MUTATIONS
        for agent in new_agents:
            if random.random() < mutation_freq:
                (i, j) = (random.randint(0, n), random.randint(0, m))
                agent[i ,j] = 1 - agent[i, j]


        # Reset
        agents = new_agents
        new_agents = []
        it += 1

    scores = [fitness(ag, context, n, m) for ag in agents]
    return(agents[np.argmax(scores)], np.max(scores))
    

print(genetic_algo("datasets/easy_dataset_1.json", 10, 0))

In [ ]:
@dataclass
class weights:
    preference: float
    authorship: float
    friendships: float

params = weights(10, 1, 1) # weitght for authorship penalty should be higher than the one for friendships


def fitness (data:dict, solution:np.array, params:weights):
    assert solution.shape == data["preferences"].shape, "Solution and preferences must have the same shape"
    assert len(solution.shape) == 2, "Solution and preferences must be 2D arrays"

    # first we compute how much the preference of each person is satisfied
    preference = np.sum(data["preferences"] * solution)
    preference = preference / np.sum(data["preferences"]) # so we get a value between 0 and 1

    # now we have to add the different penalty
    # first we compute the penalty for authorship

    authorship = np.sum(data["authorship"] * solution)
    authorship = authorship / np.sum(data["authorship"]) # so we get a value between 0 and 1

    # now we compute the penalty for friendships (I think it works )
    friendships = np.sum(data["friendships"] * solution * authorship)
    friendships = friendships / np.sum(data["friendships"]) # so we get a value between 0 and 1

    # Do we put the constraint in the fitness function or as a strcit contrainst in the genetic algorithm ? TODO think about it

    fit = params.preference * preference - (params.authorship * authorship + params.friendships * friendships)

    return fit

In [ ]:
solution = np.random.randint(0, 2, data["preferences"].shape)
fitness(data, solution, params)


4.7142857142857135